In [4]:
from requests import session,post
from pprint import pprint
from urllib.parse import urlencode
import json
import json
import time
import os

In [38]:
class OJSyncBot():
    def __init__(self):
        self.web = session()
        self.host = 'https://oj.fcu.edu.tw'
        self.web.get(self.host)
        self.table={}
        
    # 登入用副程式
    def login(self):
        acc = 'd0617735'
        password = 'Aa0512807'
        print('login',acc,"with",password,':',end='')  # 顯示登入資訊
        header = {'Host': 'oj.fcu.edu.tw',
                  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36',
                  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                  'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
                  'Accept-Encoding': 'gzip, deflate, br',
                  'DNT': '1',
                  'Connection': 'keep-alive',
                  'Upgrade-Insecure-Requests': '1',
                  'If-Modified-Since': 'Mon, 30 Dec 2019 07:09:30 GMT',
                  'If-None-Match': 'W/"5e09a2aa-569"',
                  'Cache-Control': 'max-age=0', 
                  'Referer':'https://oj.fcu.edu.tw/',
                  'Cookie': 'sessionid=ghryq3yrylcyi28kdar3mho8i6a5mx1l; _ga=GA1.3.108039447.1568633850; _gat=1; _gid=GA1.3.942735553.1577691511; csrftoken=Ptoi57U4tSIAWpjP33mQBecf8H1FyvTvFz3BLZ0n2pgBrCuojlbWqMEibTt48Rv9; FCU_WAF=egmaLkY52Dop3mjKp04iWiGx8pc0006'}                 # 一堆從FF撈出來的header...
        html = self.web.get(self.host+'/api/profile',headers=header)  # 先請求一次資料以獲取登入要用的token
        header['X-CSRFToken']=html.cookies.get('csrftoken')           # 挖出登入要用的token
        html = self.web.post(self.host+'/api/login',headers=header,data={'username':acc,'password':password})    # 把資料送到登入api
        if html.json()['data']=='Invalid username or password':                                                  # 如果登入失敗
            html = self.web.post(self.host+'/api/login',headers=header,data={'username':acc,'password':acc+acc}) # 嘗試用OJ預設密碼登入
            return html
        return html
        
    # 執行(實際上他是主程式)
    def run(self,contestID):
        contestData = self.getContestSubmissionIDs(contestID)  # 獲取所有提交紀錄
        print('Contest Submission geted')
        '''
        self.gotoDir('%d'%(contestID))
        cmdList = []
        for PID in contestData:                     # 對於比賽內所有problomID
            path = '%d/%s'%(contestID,PID)
            self.gotoDir(path)
            for NID in contestData[PID]:            # 對由該題有提交紀錄的使用者
                for idx,submit in enumerate(contestData[PID][NID]['code']):
                    filename = '%s_%03d_%s'%(NID,idx+1,submit['result'])
                    contestData[PID][NID]['fileName'].append(filename)
                    code,ip,submit = self.getSubmission(submit['id'])
                    contestData[PID][NID]['code'][idx]['code'] = submit
        '''
        with open('contestData_%d.json'%(contestID),mode='w',encoding='utf8') as f:
            json.dump(contestData,f)
            '''
            with open(path+'/'+filename+'.py','w',encoding='utf8') as f:
                f.write('# %s\n'%NID)
                f.write('## 第 %03d 次提交(%s)\n'%(idx+1,submit['time']))
                f.write('## 評測結果:%s\n'%(submit['result']))
                f.write('## 繳交IP:%s\n\n'%(ip))
                f.write(code)
            '''
            '''
            for idx in range(1,len(contestData[PID][NID]['fileName'])):
                last = contestData[PID][NID]['fileName'][idx-1]
                now = contestData[PID][NID]['fileName'][idx]
                base = 'html/'+path+'/'+NID+'_'+str(len(contestData[PID][NID]['fileName']))
                if not os.path.exists(base):
                    os.mkdir(base)
                html = base+'/'+now+'.html'
                file1 = path+'/'+last+'.py'
                file2 = path+'/'+now+'.py'
                cmd = 'python diff2HtmlCompare/diff2HtmlCompare.py "%s" "%s" "%s"'%(html,file1,file2)
                cmdList.append(cmd)
        print("Total command:",len(cmdList))
        one = len(cmdList)//100
        last = -1
        for idx,cmd in enumerate(cmdList):
            if idx//one > last:
                last += 1
                print('%d%%(%d/%d)...'%(last,idx,len(cmdList)))
            os.system(cmd)
        '''
                    
                        
                    
    def gotoDir(self,path):
        if not os.path.exists(path):        # 如果該比賽資料夾不存在
            os.mkdir(path)              # 建立資料夾
        if not os.path.exists('html/'+path):            # 如果該比賽資料夾不存在
            os.makedirs('html/'+path)                      # 建立資料夾
    
    # 將OJ的JudgeCode轉成標籤的副程式
    def getJudge(self,JudgeCode):
        if JudgeCode ==-1:
            return 'Wrong Answer'
        elif JudgeCode ==-2:
            return 'Compile Error'
        elif JudgeCode == 0:
            return 'Accepted'
        elif JudgeCode ==1:
            return 'Time Limit Exceeded'
        elif JudgeCode ==3:
            return 'Memory Limit Exceeded'
        elif JudgeCode ==4:
            return 'Runtime Error'
        elif JudgeCode ==5:
            return 'System Error'
        elif JudgeCode ==8:
            return 'Partial Accepted'
        else:
            return '未知的錯誤，錯誤代碼:%d'%(JudgeCode)
        
    # 獲取繳交時間
    def getSubmitTime(self,timeString):
        date = timeString.split('T')[0].replace('-','/') # 把日期切出來
        time = timeString.split('T')[1].split('.')[0]    # 把時間切出來
        return date+' '+time
        
    # 獲取所有的SubmissionID
    def getContestSubmissionIDs(self,ContestID,JudgeType=''):
        offset = 0               # 從第0筆開始
        limit = 200              # 一次抓200筆SubmitID
        contestData = {}         # 清空題號
        while True:
            url = self.host+'/api/contest_submissions?contest_id=%d&limit=%d&offset=%d'%(ContestID,limit,offset)  # 組合獲取0~199筆的數據的url
            if JudgeType!='':
                url+='&result='+str(JudgeType)
            html = self.web.get(url)                        # 獲取0~199筆數據
            result = html.json()['data']['results']         # 從json的是挖出我們要的資料
            totalSubmit = html.json()['data']['total']      # 獲取該比賽總提交數
            for submit in result:                           # 對於所有的提交紀錄ID
                data = {'id':submit['id'],'result':self.getJudge(submit['result']),'time':self.getSubmitTime(submit['create_time'])} # 紀錄ID、結果、繳交時間
                code,ip,submitData = self.getSubmission(submit['id'])
                data['code'] = code
                data['ip'] = ip
                if submit['result']==-2:
                    data['errMsg'] = submitData['statistic_info']['err_info']
                if submit['problem'] not in contestData:                             # 如果這個題號還沒被記錄過
                    contestData[submit['problem']]={}                                    # 把它紀錄下來
                if submit['username'] not in contestData[submit['problem']]:         # 如果這個username沒出現在這題的紀錄內
                    contestData[ submit['problem'] ][ submit['username'] ] = {'code':[],'fileName':[]}          # 將這個人新增到清單內
                contestData[ submit['problem'] ][ submit['username'] ]['code'].insert(0,data)# 把他的繳交紀錄塞到清單內
            if limit+offset >= totalSubmit:                  # 如果目前的獲取範圍 >= 總提交數
                break                                            # 終止運行
            else:                                            # 否則
                offset = limit+offset                            # 把獲取範圍往後移(即原本從0算200提交數變成從200算200提交數)
        return contestData                                   # 全部彙整完後回傳繳交紀錄
    
    def getSubmission(self,token):
        url = self.host+'/api/submission?id='+token          # 由tokren組url
        html = self.web.get(url).json()                      # 跟api換資料
        code = html['data']['code']                          # 挖出code
        ip = html['data']['ip']
        #result = html['data']['rseult']
        return code,ip,html['data']                                    # 回傳
    
    
    
    def contestInformation(self,ContestID):
        url = self.host+'/api/contest?id=%d'%(ContestID)  # 組合獲取0~199筆的數據的url
        html = self.web.get(url)                        # 獲取0~199筆數據
        result = html.json()['data']         # 從json的是挖出我們要的資料
        title = result['title'].replace('Python ','')
        
        url = self.host+'/api/contest/problem?contest_id=%d'%(ContestID)  # 組合獲取0~199筆的數據的url
        html = self.web.get(url)                        # 獲取0~199筆數據
        result = html.json()['data']         # 從json的是挖出我們要的資料
        problem={}
        for ele in result:
            problem[ele['_id']]=ele['title']
        return {'title':title,'problem':problem}

In [39]:
robot = OJSyncBot() # 召喚他
robot.login()       # 登入
problemData={}
for i in [133]:
    problemData[str(i)] = robot.contestInformation(i)       #然後獲取Contest65資料
    robot.run(i)
print(problemData)
with open('test.json','w',encoding='utf-8') as f:
    json.dump(problemData,f)

login d0617735 with Aa0512807 :Contest Submission geted
{'133': {'title': 'python_midterm_2', 'problem': {'2': '三角形', '3': '數數字', '5': '找不同、找相同', '7': '中位數', '8': '格式解構'}}}


In [21]:
with open('html/json/contestProblem.json','w',encoding='utf-8') as f:
    json.dump(problemData,f)

FileNotFoundError: [Errno 2] No such file or directory: 'html/json/contestProblem.json'